In [ ]:
from google.colab import drive
import os
import torch
from ultralytics import YOLO
from roboflow import Roboflow
from IPython.display import Image, display
from PIL import Image as PILImage
import matplotlib.pyplot as plt
import glob

In [ ]:
drive.mount('/content/drive')

project_folder = '/content/drive/MyDrive/YOLO_Pothole_Results'
os.makedirs(project_folder, exist_ok=True)

In [ ]:
!pip install ultralytics roboflow -q

In [ ]:
rf = Roboflow(api_key="YOUR_ROBOFLOW_API_KEY")

project = rf.workspace("aegis").project("pothole-detection-i00zy")
dataset = project.version(10).download("yolov8")

dataset_path = dataset.location

In [ ]:
model_checkpoint_path = '/content/drive/MyDrive/YOLO_Pothole_Results/pothole_detector_yolov8m_final/weights/last.pt'

if os.path.exists(model_checkpoint_path):
    print("Resuming training from last checkpoint...")
    model = YOLO(model_checkpoint_path)
else:
    print("Starting new training from pretrained yolov8m.pt...")
    model = YOLO('yolov8m.pt')


results = model.train(
    data=os.path.join(dataset_path, 'data.yaml'),
    epochs=200,
    imgsz=640,
    project=project_folder,       
    name='pothole_detector_yolov8m_final',
    exist_ok=True,              
    lr0=0.01,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    box=7.5,
    cls=0.5,
    dfl=1.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    bgr=0.0,
    mosaic=1.0,
    mixup=0.0,
    cutmix=0.0,
    copy_paste=0.0,
    device=0                    
)

In [ ]:

results_path = os.path.join(project_folder, 'pothole_detector_yolov8m_final')


print("Confusion Matrix:")
display(Image(filename=os.path.join(results_path, 'confusion_matrix.png'), width=800))


print("\nTraining/Validation Results:")
display(Image(filename=os.path.join(results_path, 'results.png'), width=1000))


print("\nPrecision-Recall Curve:")
display(Image(filename=os.path.join(results_path, 'PR_curve.png'), width=800))

In [ ]:
best_model_path = os.path.join(results_path, 'weights/best.pt')
model = YOLO(best_model_path)

metrics = model.val()
print("\n--- Final Model Metrics ---")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"Precision: {metrics.box.p[0]:.4f}")
print(f"Recall: {metrics.box.r[0]:.4f}")